In [6]:
from pydub import AudioSegment
import os

orig_folder = "bird_voices"
mod_folder = "bird_voices_mod"
f_format = "mp3"
mp3_files = [f for f in os.listdir(orig_folder) if f.lower().endswith(".mp3")]

for mp3_file in mp3_files:

    f = AudioSegment.from_file(
        os.path.join(orig_folder, mp3_file),
        f_format
    )

    # оставляем часть файла с 3 секунды без голоса диктора
    f = f[3000:]

    f.export(
        os.path.join(mod_folder, mp3_file),
        f_format
    )

print("Успешная обрезка аудио")

Успешная обрезка аудио


In [11]:
import os
import torchaudio
import numpy as np
from tqdm import tqdm
import librosa
import torch

# Настройки
input_dir = "bird_voices_mod"      # папка с обрезанными аудиофайлами
output_dir = "mel_data"      # куда сохранять мел-спектрограммы
segment_length = 5.0         # длина сегмента в секундах
sample_rate = 16000          # частота дискретизации
n_mels = 64                  # число мел-фильтров

os.makedirs(output_dir, exist_ok=True)

# Функция для разбиения и преобразования
def process_audio(file_path, label):
    y, sr = librosa.load(file_path, sr=sample_rate, mono=True)
    waveform = torch.tensor(y).unsqueeze(0)  # [1, time]

    total_duration = waveform.shape[1] / sample_rate
    num_segments = int(total_duration // segment_length)
    if num_segments == 0:
        num_segments = 1  # короткие файлы

    mel_transform = torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate, n_mels=n_mels
    )

    label_dir = os.path.join(output_dir, label)
    os.makedirs(label_dir, exist_ok=True)

    for i in range(num_segments):
        start = int(i * segment_length * sample_rate)
        end = int((i + 1) * segment_length * sample_rate)
        segment = waveform[:, start:end]

        if segment.shape[1] < sample_rate * 1:
            continue  # пропускаем совсем короткие куски

        mel = mel_transform(segment)
        mel_db = torchaudio.functional.amplitude_to_DB(mel, multiplier=10.0, amin=1e-10, db_multiplier=0.0)

        # Нормализация
        mel_db = (mel_db - mel_db.mean()) / (mel_db.std() + 1e-6)

        base = os.path.splitext(os.path.basename(file_path))[0]
        out_path = os.path.join(label_dir, f"{base}_{i}.npy")
        np.save(out_path, mel_db.numpy())

# Основной цикл
for fname in tqdm(os.listdir(input_dir)):
    if not fname.endswith(".mp3"):
        continue
    label = fname.split('_')[0]  # например "sparrow_001.mp3" → "sparrow"
    fpath = os.path.join(input_dir, fname)
    process_audio(fpath, label)

print("Обработка завершена! Все сегменты сохранены в mel_data")


  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:24<00:00,  2.02s/it]

Обработка завершена! Все сегменты сохранены в mel_data
